In [1]:
# readonly field

class Field:
    def __init__(self, name, type_, default=None):
        self.name = name
        self.type_ = type_
        self.value = default
        
    def __get__(self, obj, objtype):
        '''对于 Table 而言，这里的 obj 为相应的 Table 实例'''
        return self.value

    def __set__(self, obj, value):
        self.value = value

class ReadOnlyField(Field):
    def __set__(self, obj, value):
        raise Exception("readonly field")

class Table:
    d = Field('d', int, 1)
    rd = ReadOnlyField('rd', int, 2)
    
d = Table()
print(d.d) # 1
print(d.rd) # 2

d.d = 100
print(d.d) # 100

try:
    d.rd = 1111 # Exception, readonly
except Exception as err:
    print(err)

1
2
100
readonly field


In [2]:
# StaticMethod and ClassMethod

class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f
        
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args, **kwargs):
            return self.f(klass, *args, **kwargs)
        return newfunc
    
    
class A:
    
    def c_def(cls):
        print(cls)
        
    def s_def(i):
        print(i)
        
    c_def = ClassMethod(c_def)
    s_def = StaticMethod(s_def)
    
a = A()

a.c_def()
a.s_def("val")

<class '__main__.A'>
val


In [5]:
# property

class C:
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        print("get x")
        return self._x

    @x.setter
    def x(self, value):
        print("set x")
        self._x = value

    @x.deleter
    def x(self):
        print("del x")
        del self._x
        
c = C()
c.x = 1

print(c.x)

del c.x

class Foo:
    def __init__(self):
        self._x = 100000

    @property
    def bar(self):
        """readonly"""
        return self._x

foo = Foo()
print(foo.bar)

try:
    foo.bar = 111
except Exception as err:
    print(err)

set x
get x
1
del x
100000
can't set attribute


In [1]:
# example
_missing = object()


class cached_property(property):

    """A decorator that converts a function into a lazy property.  The
    function wrapped is called the first time to retrieve the result
    and then that calculated result is used the next time you access
    the value::
        class Foo(object):
            @cached_property
            def foo(self):
                # calculate something important here
                return 42
    The class has to have a `__dict__` in order for this property to
    work.
    """

    # implementation detail: A subclass of python's builtin property
    # decorator, we override __get__ to check for a cached value. If one
    # choses to invoke __get__ by hand the property will still work as
    # expected because the lookup logic is replicated in __get__ for
    # manual invocation.

    def __init__(self, func, name=None, doc=None):
        self.__name__ = name or func.__name__
        self.__module__ = func.__module__
        self.__doc__ = doc or func.__doc__
        self.func = func

    def __set__(self, obj, value):
        obj.__dict__[self.__name__] = value

    def __get__(self, obj, type=None):
        if obj is None:
            return self
        value = obj.__dict__.get(self.__name__, _missing)
        if value is _missing:
            value = self.func(obj)
            obj.__dict__[self.__name__] = value
        return value


class A:
    @cached_property
    def func(self):
        print('do something')
        return 42

a = A()
print(a.func)
print(a.func)

do something
42
42
